# Lab 01 Data Cleaning and Exploration

In this Lab, we will work with official COVID19 case data published by the Swiss government. The data can be found [here](https://github.com/openZH/covid_19/tree/master#swiss-cantons-and-principality-of-liechtenstein-unified-dataset).

It has been partially adapted from [this](https://datagy.io/pandas-data-cleaning/) tutorial.

**Note:** We will mark your tasks with 🚨 emoji. In this lab the first such task will appear relatively late.

## Fetching the Raw Data

First, we will download the raw data. Don't worry; you do not need to understand this command. The data will be saved as `swiss_covid_data.csv` in your current working directory.

In [ ]:
!wget https://raw.githubusercontent.com/vodezhaw/mldm_hs23/main/resources/lab01_dirty_data.csv -O swiss_covid_data.csv

Next, we try to inspect the downloaded data to get an understanding of its format. The command below will display the first 6 lines of the downloaded file.

Even though `.csv` stands for "comma separated values", often times people distribute `.csv` files with different separator characters. In this case, everything seems to be in order.

In [ ]:
!head -n 6 swiss_covid_data.csv

In this lab, we will use the `pandas` library to analyze and clean this dataset. This library is particularly well suited to work with tabular data.

In [ ]:
import pandas as pd  # we import the library with the short-name `pd`, a convention you will find everywhere online

Next, we will read the dataset:

In [ ]:
covid_data = pd.read_csv('swiss_covid_data.csv')

Note that if you ever work with a `.csv` that does not include the column names in the first line, or has a different separating character, you can specify these in `pd.read_csv` with the `sep` and `header` keyword arguments. For more options, you can refer to the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

The data is loaded as a `pd.Dataframe` which exposes many useful methods for this lab.

## Inspecting the Data

We can use the `.head()` method to get a quick idea of the kind of data we are dealing with. It will return the first `n=5` rows of the dataframe.

In [ ]:
covid_data.head()

Next, it is important to get a good sense what the different columns represent. Some of them have intuitive names, such as `date` and `time`, but names like `ncumul_conf` can be inscrutable.

For convenience, we copied the [data description](https://github.com/openZH/covid_19/blob/master/README.md#swiss-cantons-and-principality-of-liechtenstein-unified-dataset) below.
Be sure to study it carefully. Understanding the meaning of the data you are working with is crucial when deciding how to deal with missing values and other aspect of data cleanup!


**Metadata**

| Field Name          | Description                                | Format     | Note |
|---------------------|--------------------------------------------|------------|------|
| __date__              | Date of notification                       | YYYY-MM-DD | |
| __time__                 | Time of notification                       | HH:MM      | |
| __abbreviation_canton_and_fl__  | Abbreviation of the reporting canton       | Text       | |
| __ncumul_tested__      | Reported number of tests performed as of date| Number     | Irrespective of canton of residence |
| __ncumul_conf__          | Reported number of confirmed cases as of date| Number     | Only cases that reside in the current canton |
| __new_hosp__        | new hospitalisations since last date | Number     | Irrespective of canton of residence |
| __current_hosp__       | Reported number of hospitalised patients on date | Number     | Irrespective of canton of residence |
| __current_icu__       | Reported number of hospitalised patients in ICUs on date| Number     | Irrespective of canton of residence |
| __current_vent__        | Reported number of patients requiring invasive ventilation on date | Number     | Irrespective of canton of residence |
| __ncumul_released__     |Reported number of patients released from hospitals or reported recovered as of date| Number     | Irrespective of canton of residence |
| __ncumul_deceased__     |Reported number of deceased as of date| Number     | Only cases that reside in the current canton |
| __source__              | Source of the information                  | href       | |
| __current_isolated__       | Reported number of isolated persons on date          | Number       | Infected persons, who are not hospitalised |
| __current_quarantined__    | Reported number of quarantined persons on date       | Number       | Persons, who were in 'close contact' with an infected person, while that person was infectious, and are not hospitalised themselves |
| __current_quarantined_riskareatravel__    | Reported number of quarantined persons on date       | Number       | People arriving in Switzerland from [certain countries and areas](https://www.bag.admin.ch/bag/en/home/krankheiten/ausbrueche-epidemien-pandemien/aktuelle-ausbrueche-epidemien/novel-cov/empfehlungen-fuer-reisende/quarantaene-einreisende.html), who are required to go into quarantine.  |

Another useful method to get a quick feel for the data at hand is `.describe()` which produces a table of statistics about the different columns. By default it will only include numeric columns, but you can set `include='all'` to get an overview of all columns. Note that when computing statistics such as the mean, `NaN` values will be ignored.

In [ ]:
covid_data.describe(include='all')
# try setting `include='all'`

## Dealing with Missing Values

When reading the original `swiss_covid_data.csv` every cell that did not have a value was assigned the value `NaN` (not a number).

When we apply the `.isna()` method to our data, every cell of the result will contain a boolean value indicating whether that cell had a `Nan` (or `None` or similar false-y value) in it.

In [ ]:
covid_data.isna()

We can count the number of missing values in each column by summing over all rows:

In [ ]:
covid_data.isna().sum()

*Note*: You can specify `.sum(axis=1)` to sum over columns instead.

We notice that the `current_quarantined_total` has 24040 missing values, which corresponds to the total number of entries in the dataframe.

We can re-verify this by inspecting the unique values:

In [ ]:
covid_data['current_quarantined_total'].unique()

We therefore suggest to drop this column from the data entirely.

This can be achieved by the `.drop()` method, as follows:

In [ ]:
covid_data = covid_data.drop(columns=['current_quarantined_total'])
covid_data.head()

Generally, there are two approaches to deal with missing values. We can either ignore them, for example by throwing out rows that contain them, or we can try to impute a "reasonable" value.

Of course, what is "reasonable" heavily depends on the data at hand!

Let us first consider the easier approach: dropping rows that have missing entries. This can be achieved using the `.dropna()` method.

By default it will remove all rows that have at least one missing value:

In [ ]:
covid_data.dropna()

We can see that this is too aggressive as it leaves us with only 68 out our original 24032 rows!

We can be more lax about dropping rows by either setting `how='all'`, which will only drop rows that consist entirely of missing values, or `thresh=3` (or any other integer value) which will drop rows with fewer than 3 non-missing values.

In [ ]:
covid_data.dropna(thresh=3)

### Replacing Missing Values

We will now focus on imputing missing values.

Our data contains broadly speaking 3 types of values:
* direct and cumulative counts of populations (numerical)
* `date` and `time`
* categorical `abbreviation_canton_and_fl` and `source`

We already know that `abbreviation_canton_and_fl` and `date` do not have any missing values.

`source` has only 2 missing values, which we will ignore for now.

We will first focus on the `time` column. Looking at the dataframe console printouts, we can already guess that many rows have the value `"00:00"` as their time.

We can confirm this by counting how many times each unique `time` value appears:

In [ ]:
covid_data['time'].value_counts()

We will therefore decide to fill the missing `time` values with
`"00:00"`. This can be achieved by the `.fillna()` method:

In [ ]:
covid_data['time'] = covid_data['time'].fillna("00:00")
covid_data.isna().sum()

Note that we specifically update the `time` column with a version where the missing values have been replaced. If we instead wrote `covid_data.fillna("00:00")` then every missing value in the entire dataframe would be replaced by `"00:00"`!

**(Side Note)**: If we inspect the data more closely, we can see that each canton seems to have a specific time that they usually submit the data, so filling in "00:00" agnostically might not be the perfect solution but we will consider it good enough for this lab.

Next, we will fill in current counts, such as `current_hosp`. In the lecture you have seen different approaches to determine a good value to impute. Intuitively, if we do not know how many people are currently in the hospital, a simple estimate would be to use the number of people hospitalised the day before, or the last known value in general.

This can be achieved in pandas by using the `.ffill()` (forward-fill) method. The code sniplet below shows a simple example.

In [ ]:
ffill_example = pd.DataFrame({"values": [1, 2, 3, None, None, 6]})
ffill_example

In [ ]:
ffill_example.ffill()

For our covid data this will become a little more intricate. First, we have to make sure that the data is sorted by date and time to make sure we actually forward fill the last known values. Second, we will have to do this for each canton individually, as it makes no sense to substitute a known value from Geneva in Zurich. We show how to achieve this in the next cell and unpack it afterwards:

In [ ]:
ffill_columns = [
  'ncumul_tested',
  'ncumul_conf',
  'current_hosp',
  'current_icu',
  'current_vent',
  'ncumul_released',
  'ncumul_deceased',
  'current_isolated',
  'current_quarantined',
  'current_quarantined_riskareatravel',
]
for canton in covid_data['abbreviation_canton_and_fl'].unique():
  covid_data.loc[covid_data['abbreviation_canton_and_fl'] == canton, ffill_columns] = covid_data.loc[covid_data['abbreviation_canton_and_fl'] == canton, ffill_columns].ffill().fillna(0.)

covid_data

First, we specify which columns we will forward fill. Here we selected all current and cumulative counts.

Then we iterate over all cantons. The `.loc` method is used for complex indexing. It can be used as `.loc[row_selection, column_selection]`. The rows we select are the ones that correspond to the current canton. The expression `covid_data['abbreviation_canton_and_fl'] == canton` returns a boolean index series that indicates rows that match the current canton. We use the list of columns we want to forward fill as our column selection.

As before, we reassign all these cells by a version that has its missing values filled in. We apply the `.ffill()` method and follow it by a `fillna(0.)`. This is because missing values at the start (before any value is known) can not be forward-filled and we explicitely set them to 0 here.

Let us now see our progess:

In [ ]:
covid_data.isna().sum()

There are two remaining columns with missing values. For `new_hosp` we will naively fill in 0 and for `source` we will impute the string `"unknown"`.

**🚨 TASK 1A (2 Points) 🚨**

* Replace missing values in the `new_hosp` column by 0
* Replace missing values in the `source` column by the string `"unknown"`

In [ ]:
# Hint refer back to where we replaced missing 'time' values by "00:00"!
# TODO
covid_data['new_hosp'] = covid_data['new_hosp'].fillna(0)
covid_data['source'] = covid_data['source'].fillna("unknown")

We have now eliminated all missing values!

In [ ]:
covid_data.isna().sum()

<mark>In **MOODLE**</mark>:

* upload your code snippte to fill in missing values for `new_hosp` and `source`
* upload the output of counting remaining missing values

## Dealing with Duplicate Rows

The `.duplicated()` method will tell you for each row whether it is an exact duplicate. By default it will mark the first occurrence of a row as a non-duplicate and every following occurrence as a duplicate.

In [ ]:
covid_data.duplicated()

We can check whether there are any exact duplicates by summing:

In [ ]:
covid_data.duplicated().sum()

We can see that our dataset does not contain any duplicate columns, this is mainly because each entry has a unique `'date'` and `'time'`.

We can set the `subset=...` parameter to indicate which columns to consider for duplicate detection. Let us check whether there are duplicates, when we ignore the `'date'` and `'time'` columns:

**🚨 TASK 1B (1 Point) 🚨**

* count the number of exact duplicates when we only consider non-time colums

In [ ]:
non_time_cols = [c for c in covid_data.columns if c not in {'date', 'time'}]

In [ ]:
# TODO
covid_data.duplicated(subset=non_time_cols).sum()

<mark>In **MOODLE**</mark>:
* upload the number of exact duplicates considering only non-time columns

There is a good chance that the duplicates are a result of our missing value imputation approach.

Here it does not seem like a good idea to drop any rows as those rows that have identical entries except date and time are not a-priori problematic.


Nevertheless, it is good to know how to remove duplicate rows for future reference.

We can do so using the `.drop_duplicates()` method, which will drop those rows that `.duplicated()` indicates as duplicates:

Don't worry about running the next cell, it will return a new dataframe with rows dropped but will not change your current version of `covid_data`.

In [ ]:
covid_data.drop_duplicates(subset=non_time_cols)

## Cleaning up the 'source' Column

Looking at the values in the `source` column, we can see that they correspond to URLs. Many of these belong to the same domain and only vary in their parameters.

🚨 **TASK 1C (1 Point)** 🚨

* inspect the unique values in the `source` column
* count the unique values in the `source` column

In [ ]:
# Hint: we have already done this for different column earlier
len(covid_data['source'].unique())

In [ ]:
# the `.unique` method returns a numpy array, you can use the built-in `len` function to check its length

<mark>In **MOODLE**</mark>:
* report the number of unique values in the `source` column.

Let us try making this data more digestable by extracting the domain information.

For this we will use the following helper function:

In [ ]:
from urllib.parse import urlparse
def extract_netloc(url):
  return urlparse(url).netloc

# lets try it out
url = 'https://www.baselland.ch/politik-und-behorden/direktionen/volkswirtschafts-und-gesundheitsdirektion/amt-fur-gesundheit/medizinische-dienste/kantonsarztlicher-dienst/aktuelles/covid-19-faelle-kanton-basel-landschaft'
print(extract_netloc(url))

We can apply this function to the `source` column to replace it with a cleaned version:

In [ ]:
covid_data['source'] = covid_data['source'].apply(extract_netloc)
covid_data

In [ ]:
sorted(covid_data['source'].unique())

The `.apply` method applies the function passed as an argument to each element.

Let's now save the final cleaned dataset:

In [ ]:
covid_data.to_csv('swiss_covid_data_clean.csv')

At this point, we consider the data cleaning process done.

ℹ️ **In the next cell, we will download a reference version of the cleaned data that you can use, if you did not manage to implement part of the process so far.**

In [ ]:
!wget https://raw.githubusercontent.com/vodezhaw/mldm_hs23/main/resources/lab01_clean_data.csv -O lab01_clean_data.csv

In [ ]:
# uncomment the next line, if you want to use reference data
covid_data = pd.read_csv('lab01_clean_data.csv')

## Using the Clean Data

### **🚨 TASK 2 (3 Points) 🚨**

Now that you have a clean dataset, it is time for some data exploration.

* think of a question that you could try to answer with the given dataset
* use summary statistics and/or visualizations to try to answer your question

<mark>On **Moodle**</mark> _upload a <mark>PDF</mark>_ answering the following questions:
* What was your initial question or idea?
* How did you proceed to arrive at an answer?
* What are your results?
* Include code-snippets, plots, and similar to support your answer.

Below, we include some code that helps you plot time-dependent variables.
Note that **it is _NOT_ mandatory to produce any plots for this assignment**!

In [ ]:
# You might want to plot things
from matplotlib import pyplot as plt

In [ ]:
# replace 'date' string by parsed datetime objects
# this can help if you want to use matplotlib to create a time plot
# you do not have to understand this
from datetime import datetime as dt
covid_data['date'] = pd.to_datetime(covid_data['date'].apply(lambda t: dt.strptime(t, "%Y-%m-%d")))

In [ ]:
# extract dates for Geneva
x = covid_data.loc[covid_data['abbreviation_canton_and_fl'] == 'GE', ['date']]
# extract the 'ncumul_test' column for Geneva
y = covid_data.loc[covid_data['abbreviation_canton_and_fl'] == 'GE', ['ncumul_tested']]

# plot
plt.plot_date(x, y)

In [ ]:
# Generic scatter-plot that does not include a time variable
plt.scatter(covid_data['ncumul_tested'], covid_data['ncumul_deceased'])

# TASK 2
My idea is to show how the numbers of tested persons, 
positive persons and hospitalized persons differ in French speaking Switzerland, German speaking Switzerland and Italian speaking Switzerland.

In [ ]:
num_habitants_fr = 839071
num_habitants_ge = 1595334
num_habitants_it = 356674

# French speaking Switzerland ncumul_tested
french_speaking_cantons = ['VD']
filtered_fr_data = covid_data[covid_data['abbreviation_canton_and_fl'].isin(french_speaking_cantons)]

# German speaking Switzerland ncumul_tested
german_speaking_cantons = ['ZH']
filtered_ge_data = covid_data[covid_data['abbreviation_canton_and_fl'].isin(german_speaking_cantons)]


# Italian speaking Switzerland ncumul_tested
italian_speaking_cantons = ['TI']
filtered_it_data = covid_data[covid_data['abbreviation_canton_and_fl'].isin(italian_speaking_cantons)]


In [ ]:
# current hospitalized per habitant

x_fr = filtered_fr_data['date']
x_ge = filtered_ge_data['date']
x_it = filtered_it_data['date']
y_fr = filtered_fr_data['current_hosp'] / num_habitants_fr
y_ge = filtered_ge_data['current_hosp'] / num_habitants_ge
y_it = filtered_it_data['current_hosp'] / num_habitants_it

# plot
plt.scatter(x_fr, y_fr, s=5, label='French Speaking')
plt.scatter(x_ge, y_ge, s=5, label='German Speaking')
plt.scatter(x_it, y_it, s=5, label='Italian Speaking')
plt.xlabel('Date')
plt.ylabel('Cumulative Tested')
plt.title('Cumulative Tested per habitant vs. Date for Selected Canton')
plt.legend()
plt.show()



In [ ]:
# current in icu

x_fr = filtered_fr_data['date']
x_ge = filtered_ge_data['date']
x_it = filtered_it_data['date']
y_fr = filtered_fr_data['current_icu'] / num_habitants_fr
y_ge = filtered_ge_data['current_icu'] / num_habitants_ge
y_it = filtered_it_data['current_icu'] / num_habitants_it

# plot
plt.scatter(x_fr, y_fr, s=5, label='French Speaking')
plt.scatter(x_ge, y_ge, s=5, label='German Speaking')
plt.scatter(x_it, y_it, s=5, label='Italian Speaking')
plt.xlabel('Date')
plt.ylabel('Cumulative in icu ')
plt.title('current in icu per habitant vs. Date for Selected Canton')
plt.legend()
plt.show()


In [ ]:
#current needed vent

x_fr = filtered_fr_data['date']
x_ge = filtered_ge_data['date']
x_it = filtered_it_data['date']
y_fr = filtered_fr_data['current_vent'] / num_habitants_fr
y_ge = filtered_ge_data['current_vent'] / num_habitants_ge
y_it = filtered_it_data['current_vent'] / num_habitants_it

# plot
plt.scatter(x_fr, y_fr, s=5, label='French Speaking')
plt.scatter(x_ge, y_ge, s=5, label='German Speaking')
plt.scatter(x_it, y_it, s=5, label='Italian Speaking')
plt.xlabel('Date')
plt.ylabel('current vent needed per habitant')
plt.title('current vent needed per habitant vs. Date for Selected Canton')
plt.legend()
plt.show()
